# Computer Vision 152
### Binarization, Connected Components, Filtering
##### Catherine Shen

## Problem 1: Binarization

Write a Python function to implement Otsu’s method described in class. This algorithm should automatically determine an intensity level to threshold an image to segment out the foreground from the background. The output of your function should be a binary image that is black (pixel values = 0) for all background pixels and white (pixel values = 1) for all foreground pixels. Apply this function to the image can_pix.png and turn in the output image in your report.

Notes:
 - Load in an image and convert it to grayscale.
 - You can use np.histogram or matplotlib.pyplot.hist to create a histogram of pixel intensities as a first step.


## 1  
Otsu's method produces a binary image to separate the foreground (cans) from the background. Image pixels are labeled 0 (black for background) if they are less than the calculated threshold, else they are labeled 1 (white for foreground). The threshold was calculated from the histogram data of the greyscale image. Since there are 256 levels of intensity, the number of bins was set to 256. Decreasing the number of bins would decrease the resolution of the resulting binary image.  
  
Greyscale image of can_pix.png  
<img src="p1_greyimg.png" />  
  
Histogram of image can_pix.png. The x-axis is the bins for the 256 intensity levels and the y-axis is the number of image pixels with that intensity.  
<img src="p1_histo.png" />  
  
Binary image of can_pix.png  
<img src="p1_binimg.png" />  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc
#import imageio

#Parameters: grey_img - greyscale img
def otsu(grey_img):
    #Total pixels
    img_size = np.size(grey_img)
    print('img size: ', img_size)
    row, col = np.shape(grey_img)
    print(str(row) + 'x' + str(col))
    
    binary_img = np.copy(grey_img)
    
    #Create new figure. Prevent overlap of figures
    plt.figure()
    #Typo - only 2 return values instead of 3
    #h, bins = plt.hist(grey_img)   # Histogram of px intensities
    #Or can use np.histogram, can ignore patches
    #Ravel to flatten img array, return normalized histogram
    #n, bins, patches = plt.hist(grey_img.ravel(), bins=256, density=True)
    #Normalizing histogram did not result in total probability = 1
    n, bins, patches = plt.hist(grey_img.ravel(), bins=256)
    #plt.savefig('p1_histo.png')
    #plt.show()
    
    #print('n: \n', n)
    #print('bins: \n', bins)
    n_length = len(n)
    print('length n: ', n_length)
    #b_length = len(bins)
    print('length bins: ', len(bins))

    # Set appropriate threshold using Otsu's method
    # Your code here
    #
    threshold = 0
    deviation = []
    #Get probabilities
    n = [n[i]/img_size for i in range(0,n_length)]
    all_sum = np.cumsum(n)
    #print('all_sum: \n', all_sum)
    all_mean = np.cumsum([j*n[j] for j in range(0,n_length)])
    #4.Global mean
    global_mean = all_mean[-1]
    #Loop through each intensity level threshold, get threshold w/ max deviation
    for i in range(0,n_length):
        #2.Calculate the cumulative sums / weights to left and right of threshold
        w0 = all_sum[i-1]
        w1 = 1-w0
        #print('w0, w1: ', w0, w1)
        #No 0 weights 
        if w0 <= 0 or w1 <= 0:
            deviation.append(0)
            continue
        #3.Calculate cumulative mean
        mean0 = all_mean[i-1]/w0
        mean1 = (all_mean[-1]-mean0)/w1
        #mean0 = all_mean[i]/w1
        #mean1 = (all_mean[-1]-mean0)/w1
        #5.Deviation
        temp_dev = w0*w1*(mean0-mean1)**2
        #temp_dev = ((global_mean*w0-w1)**2)/(w0*w1)
        deviation.append(temp_dev)
        """
        if temp_dev >= deviation:
            deviation = temp_dev
            threshold = i
        """
    #Get threshold with max deviation
    threshold = [k for k,dev in enumerate(deviation) if dev==max(deviation)][0]
    #print('deviation: ', deviation)
    print('threshold: ', threshold)
    #Output:63
    
    ####
    
    # Set pixel values in the binary_img based on the threshold
    # Your code here
    #
    for i in range(0,row):
        for j in range(0,col):
            #Black 0 if less than threshold, else white 1
            if binary_img[i][j] < threshold:
                binary_img[i][j] = 0
            else:
                binary_img[i][j] = 1
    return binary_img


#Convert image to greyscale
img = misc.imread('can_pix.png', flatten=True)
#imshow default appears green but numpy array still correct
#View in gray
plt.imshow(img, cmap = 'gray')
plt.imsave('p1_greyimg.png', img, cmap='gray')
#plt.savefig('p1_greyimg.png')
#print(img)
can_bin_img = otsu(img)
plt.figure()
plt.imshow(can_bin_img, cmap = 'gray')
plt.imsave('p1_binimg.png', can_bin_img, cmap='gray')
#plt.savefig('p1_binimg.png')

## Problem 2: Connected Components

$\textbf{2.1 Connected Regions}$

(a) Write Python code to implement the connected component labeling algorithm discussed in class, assuming 8-connectedness. Your function should take as input a binary image (computed using your algorithm from question 1) and output a 2D matrix of the same size where each connected region is marked with a distinct positive number (e.g. 1, 2, 3). On the
image can pix.png, display an image of detected connected components where connected region is mapped to a distinct color using imshow. You may need to handle recursive calls carefully to avoid system crash by using sys.setrecursionLimit(1000).

(b) How many components do you think are in the image coins pix.jpg? What does your connected component algorithm give as output for this image? Include your output on this image in your report. It may help to reduce the size of the image before running the connected components algorithm but after converting the image to a binary image.

$\textbf{2.2 Take your own images}$

For this part, you will be using images you take with your own camera. Choose three objects of different shapes, preferably with non-shiny surfaces. Possibilities include paper/cardboard cut-outs, bottle tops, pencils, Lego pieces, potatoes, etc. Take 3 pictures of these objects individually with a solid background. The object should be clearly distinguishable from the background (e.g. bright object & dark background). Include these three images in your report as in Figure 1. In addition, show similar output images as in Problem 2.1 for each of your new images (there is only 1 connected component in this case).

<img src="sample_imgs.png">
<center>Figure 1: Sample images.</center>

$\textbf{2.3 Image moments and rectification}$

Write three functions which compute the moments, central moments, and normalized moments of a marked region. Each function should take as input a 2D matrix (output of part 2.2) and 3 numbers $j$, $k$, and $d$. The output should be the $(j, k)$ moment $M_{j,k}$, central moment $\mu_{j,k}$, normalized moment $m_{j,k}$ of the region marked with positive number $d$ in the input matrix.

Using these functions, on each of the three images, draw the centroid of each object. Also
compute the eigenvectors of the centralized second moment matrix and draw the two eigenvectors
on the centroid. This should indicate the orientation of each object, see Figure 2 for the results on the example images. Turn in the outputs for your own images.

<img src="principle_dirs.png">
<center>Figure 2: Sample images with principle directions overlaid.</center>

$\textbf{2.4 Image alignment}$

We have seen that the orientation computed from Problem 2.3 can be used to roughly align the orientation of the region (i.e. in-plane rotation). Write a function to rotate the region around its centroid so that the eigenvector corresponding to the largest eigenvalue (i.e. the blue vector in Figure 3) will be horizontal (aligned with $[1, 0]^T$). This might look like in Figure 3.
Your function should take as input a 2D matrix (output of Problem 2.1) and output a matrix of the same size in which all marked regions are aligned. Turn in the aligned outputs for your images.
Note: After finding the rotation matrix R to rotate the largest eigenvector to $[1, 0]^T$ , we rotate all points $(x, y)$ belonging to that region using the following transformation

\begin{eqnarray}
\begin{bmatrix}x'\\y'
\end{bmatrix}=R\begin{bmatrix}x-\hat{x}\\y-\hat{y}
\end{bmatrix}+\begin{bmatrix}\hat{x}\\\hat{y}
\end{bmatrix}
\end{eqnarray}


where $[\hat{x}, \hat{y}]^T$
T are the centroid coordinates of the region. For simplicity, just ignore the cases when
part of the aligned region falls outside of the image border or is overlapped with other regions. You
can avoid these issues when capturing your images (e.g. put your objects a bit far apart). Finally,
note that the rotation matrix can be created trivially from the eigenvectors.

<img src="rotated.png">
<center>Figure 3: (a) Original binarized image with principal directions (b) Aligned sample image with major eigenvector aligned along $[1, 0]^T$.</center>


## 2.1a  
The number of connected components of the can_pix.png image was calculated. Six different connected components were detected. Decreasing the number of bins in the histogram would decrease the resolution of the binary image, therefore, decreasing the number of detected connected components since the cans would appear to merge.  
  
Detected connected components of cans image  
<img src="p2-1a_cc.png" />  
  
## 2.1b  
The coins_pix.png image would appear to have 12 connected components, but it actually has 2 since all but one of the coins are connected in the binary image. Decreasing the number of bins in the histogram would decrease the resolution and the number of detected connected components, while increasing the number to increase resolution would not produce much of an effect.  
  
The coins image was inverted so that the background is black and the foreground (coins) is light-colored to easily calculate the connected components.
<img src="p2-1b_inv_coins_pix.png" />  

Binary image of coins  
<img src="p2-1b_bin_coins_pix.png" />  
  
Detected connected components of coins  
<img src="p2-1b_cc_coins.png" />  
  
## 2.2  
Three objects with different shapes were chosen and their connected components were calculated. Using a black background with a light-colored object increases the accuracy of separating the foreground from the background.  
  
Tape and connected component  
<img src="p2-2_tape.jpg" />  
<img src="p2-2_cctape.png" />  

Post and connected component  
<img src="p2-2_post.jpg" />  
<img src="p2-2_ccpost.png" />  
  
Rect and connected component  
<img src="p2-2_rect.jpg" />  
<img src="p2-2_ccrect.png" />  
  
## 2.3  
Calculating the centroids and the major and minor eigenvectors and eigenvalues of each image was from calculating the moments, central moments, and normalized moments of the chosen connected component. The blue arrow is the major eigenvector that runs along the longest side of the object while the red arrow is the minor eigenvector that runs along the shortest side and is orthogonal to the major eigenvector.  
  
Tape  
<img src="p2-3_cctape.png" />  
  
Post  
<img src="p2-3_ccpost.png" />  
  
Rect  
<img src="p2-3_ccrect.png" />  
  
## 2.4  
The chosen connected component region for each image was rotated around the centroid so that the major eigenvector is parallel to the x-axis. Some pixels in the connected component after rotation appear to be missing since each connected component pixel was rotated and rounded to the nearest pixel.  
  
Tape  
<img src="p2-4_cctape.png" />  
  
Post  
<img src="p2-4_ccpost.png" />  
  
Rect  
<img src="p2-4_ccrect.png" />  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import itertools
import cv2 
import math

sys.setrecursionlimit(100000)

#Parameter: img - binarized image
def connected_component(img):
    # your code here
    cc_img = img.copy()
    row, col = np.shape(cc_img)
    #Start from 2 b/c 1 is white
    comp = 2
    #Label components
    for i in range(0,row):
        for j in range(0,col):
            neighbors = []
            #If pixel is 1 and not explored (explored will mark as comp)
            if img[i][j] == 1 and cc_img[i][j] == 1:
                #Add to neighbors and mark as explored using comp
                neighbors.append((i,j))
                cc_img[i][j] = comp
                
                #Set connected component neighbors
                while len(neighbors) != 0:
                    #Pop from front
                    x, y = neighbors.pop(0)
                    # Get 8 neighbor pixels. If not explored, add to neighbors
                    for i in range(-1,2):
                        for j in range(-1,2):
                            if (x+i)>=0 and (x+i)<row and (y+j)>= 0 and (y+j)<col:
                                #If not explored yet, add to neighbors and mark
                                if cc_img[x+i][y+j] == 1:
                                    neighbors.append((x+i,y+j))
                                    cc_img[x+i][y+j] = comp
                #label(cc_img[i][j],comp)
                comp += 1 
    #print('connected component: ', comp)
    return cc_img

#Parameters: cc_img - connected component img
#            j, k - (j,k)th moment
#            d - # of the connected region you're calculating the moment of
def moment(cc_img, j, k, d):
    # your code here
    M_jk = 0
    row, col = np.shape(cc_img)
    for x in range(0,row):
        for y in range(0,col):
            #If part of desired component region
            if cc_img[x][y] == d:
                M_jk += (x**j)*(y**k)
    return M_jk

def central_moment(cc_img, j, k, d):
    # your code here
    mu_jk = 0
    row, col = np.shape(cc_img)
    M00 = moment(cc_img,0,0,d)
    M01 = moment(cc_img,0,1,d)
    M10 = moment(cc_img,1,0,d)
    #Centroid of img
    x_avg = M10/M00
    y_avg = M01/M00
    for x in range(0,row):
        for y in range(0,col):
            if cc_img[x][y] == d:
                mu_jk += ((x-x_avg)**j)*((y-y_avg)**k)
    return mu_jk

def norm_moment(cc_img, j, k, d):
    # your code here
    m_jk = 0
    row, col = np.shape(cc_img)
    M00 = moment(cc_img,0,0,d)
    M01 = moment(cc_img,0,1,d)
    M10 = moment(cc_img,1,0,d)
    #Centroid of img
    x_avg = M10/M00
    y_avg = M01/M00
    mu20 = central_moment(cc_img,2,0,d)
    mu02 = central_moment(cc_img,0,2,d)
    x_sigma = math.sqrt(mu20/M00)
    y_sigma = math.sqrt(mu02/M00)
    for x in range(0,row):
        for y in range(0,col):
            if cc_img[x][y] == d:
                m_jk += (((x-x_avg)/x_sigma)**j)*(((y-y_avg)/y_sigma)**k)
    return m_jk

#Parameters: cc_img - connected component img
#            d - connected component #
#            plot_align - rotation/alignment
#            name - string for save img
def plot_img_moments(cc_img, d, name, plot_align=0):
    # your code here
    row, col = np.shape(cc_img)
    M00 = moment(cc_img,0,0,d)
    M01 = moment(cc_img,0,1,d)
    M10 = moment(cc_img,1,0,d)
    #Centroid of img
    x_avg = M10/M00
    y_avg = M01/M00
    centroid = np.array([[x_avg],[y_avg]])
    print('centroid: ',centroid)
    plt.imshow(cc_img)
    #Plot centroid in red
    plt.plot([y_avg],[x_avg],'ro')
    #plt.plot([x_avg],[y_avg],'ro')
    #Second centralized moment matrix
    mu11 = central_moment(cc_img,1,1,d)
    mu02 = central_moment(cc_img,0,2,d)
    mu20 = central_moment(cc_img,2,0,d)
    matrix_2nd = np.array([[mu20,mu11],
                         [mu11,mu02]])
    #Compute the major and minor eigenvectors and eigenvalues
    e_vals, e_vectors = np.linalg.eig(matrix_2nd)
    #print('eigenvectors: ', e_vectors)
    #print('eigenvalues: ', e_vals)
    #draw the two eigenvectors on the centroid
    #find longest eigenvalue and its eigenvector
    if(e_vals[0] < e_vals[1]):  
        plt.arrow(y_avg,x_avg,e_vectors[1,0]*40,e_vectors[0,0]*40,shape='full',color='red',lw=3,length_includes_head=True, head_width=.01)
        plt.arrow(y_avg,x_avg,e_vectors[1,1]*40,e_vectors[0,1]*40,shape='full',color='blue',lw=3,length_includes_head=True, head_width=.01)
    else:
        plt.arrow(y_avg,x_avg,e_vectors[1,0]*40,e_vectors[0,0]*40,shape='full',color='blue',lw=3,length_includes_head=True, head_width=.01)
        plt.arrow(y_avg,x_avg,e_vectors[1,1]*40,e_vectors[0,1]*40,shape='full',color='red',lw=3,length_includes_head=True, head_width=.01)      
    plt.title('Connected Components with two eigenvectors on centroid')
    plt.savefig('p2-3_' + name + '.png')
    plt.show()
    
    #alignment need
    if(plot_align):
        #longest eigenvalue and its eigenvector as column zero
        #if(e_vals[0] < e_vals[1]):  
        #    e_vectors_sw = e_vectors.T
        #else:
        #    e_vectors_sw = e_vectors
        #aligned_img = rotate_region(cc_img, centroid, e_vectors_sw, e_vals, d)
        aligned_img, e_vectors = rotate_region(cc_img, centroid, e_vectors, e_vals, d)
        #plt.imshow(np.rot90(aligned_img,-3))
        plt.imshow(aligned_img)
        #mu20 = central_moment(aligned_img, 2, 0, d)
        #mu02 = central_moment(aligned_img, 0, 2, d)
        #mu11 = central_moment(aligned_img, 1, 1, d)
        #compute eigenvectors of the centralized second moment matrix
        #matrix_2nd = np.array([[mu20, mu11],
        #                [mu11, mu02]])
        #e_vals, e_vectors = np.linalg.eig(matrix_2nd)
        #print("eigenvector align", e_vectors)
        #print("eigenvalue align", e_vals)
        #draw the two eigenvectors on the centroid
        #the largest eigenvector corresponding to the largest eigenvalue with blue color
        if(e_vals[0] < e_vals[1]):  
            plt.arrow(y_avg,x_avg,e_vectors[1,0]*40,e_vectors[0,0]*40,shape='full',color='red',lw=3,length_includes_head=True, head_width=.01)
            plt.arrow(y_avg,x_avg,e_vectors[1,1]*40,e_vectors[0,1]*40,shape='full',color='blue',lw=3,length_includes_head=True, head_width=.01)
        else:
            plt.arrow(y_avg,x_avg,e_vectors[1,0]*40,e_vectors[0,0]*40,shape='full',color='blue',lw=3,length_includes_head=True, head_width=.01)
            plt.arrow(y_avg,x_avg,e_vectors[1,1]*40,e_vectors[0,1]*40,shape='full',color='red',lw=3,length_includes_head=True, head_width=.01)      
        plt.title('Aligned Connected Component with two eigenvectors on centroid')
        plt.savefig('p2-4_' + name + '.png')
        plt.show()

#Parameters: cc_img - connected component img
#            centroid - 2x1 of connected component
#            eigenvectors - 2x2
#            e_vals - 1x2 eigenvalues
#            d - # of the connected region you're calculating 
def rotate_region(cc_img, centroid, eigenvectors, e_vals, d):
    #aligned_img = np.copy(cc_img)
    # your code here
    nrow, ncol = np.shape(cc_img)   
    aligned_img = np.zeros((nrow,ncol))
    x_axis = np.array([[0],[1]]).flatten()
    #u_xaxis = x_axis/np.linalg.norm(x_axis)
    #Normalize
    u_xaxis = x_axis/np.sqrt(np.dot(x_axis,x_axis))
    #[1,0].T for large eigenvector; [0,1].T for the other eigenv.
    #I = np.eye(2)                   
    #Angle between major eigenvector and x-axis
    #Unit vector
    if(e_vals[0] < e_vals[1]):  
        #Col 1
        e_vec1 = eigenvectors[:,1].flatten()
    else:
        #Col 0
        e_vec1 = eigenvectors[:,0].flatten()
    #Normalize
    #u_vec1 = e_vec1/np.linalg.norm(e_vec1)
    u_vec1 = e_vec1/np.sqrt(np.dot(e_vec1,e_vec1))
    angle = math.acos(np.dot(u_vec1, u_xaxis))
    #print('angle: ', angle)
    #Rotation matrix
    #R = np.dot(B,eigenvectors.T)    #2 x 2    
    #R = np.dot(np.linalg.inv(eigenvectors),I) #eigenvector matrix transpose is same as invert
    #R = np.array([[math.cos(angle),math.sin(angle)],[(-1)*(math.sin(angle)),math.cos(angle)]])
    R = np.eye(2,2)
    #Counter
    R[0,0] = math.cos(angle)
    R[0,1] = -1*math.sin(angle)
    R[1,0] = math.sin(angle)
    R[1,1] = math.cos(angle)
    #print("expected eigenvector", np.dot(R,eigenvectors))
    #print("expected eigenvector", np.dot(R,e_vec1))
    #print("Rotate R", R)
    #count = 0
    #Rotate img by angle
    for x in range(0, nrow):
        for y in range(0, ncol):
            if cc_img[x,y] == d:   #marked as d #all regions or just region 1 ?
                #count += 1
                pix = np.array([[x],[y]])
                aligned_pix = np.dot(R,(pix-centroid)) + centroid  #2 x 1
                #aligned_img[int(aligned_pix[0]),int(aligned_pix[1])] = cc_img[x,y]  #i.e. 1
                aligned_img[int(np.round(aligned_pix[0])),int(np.round(aligned_pix[1]))] = cc_img[x,y]  #i.e. 1
                #Set as 'marked'
                #aligned_img[x,y] = 0
                #print("alinged", x, y, aligned_pix[0], aligned_pix[1])        
    #print("tot aligned pixels", count)
    #Rotate eigenvectors
    eigenvectors = np.dot(R,eigenvectors)
    return aligned_img, eigenvectors
    
# Part 2.1a
#can_img = misc.imread('p1_binimg.png')
#plt.imshow(can_bin_img)
#plt.figure()
#can_bin_s = cv2.resize(can_bin_img)
cc_can = connected_component(can_bin_img)
#print('Connected components of cans')
#plt.imshow(cc_can)
plt.imsave('p2-1a_cc.png',cc_can)

# Part 2.1b
plt.figure()
img = misc.imread('coins_pix.jpg')
#Flip every pixel to get black/dark background
img = 1 - np.asarray(img,)
#print('Inversed coins img')
#plt.imshow(img)
plt.imsave('p2-1b_inv_coins_pix.png',img)
#Convert image to greyscale
coin_grey = misc.imread('p2-1b_inv_coins_pix.png', flatten=True)
coin_bin = otsu(coin_grey)
#Make sure that histogram and bin images don't ouput with overlap
plt.figure()
#print('Binarized coins img')
#plt.imshow(coin_bin, cmap='gray')
plt.imsave('p2-1b_bin_coins_pix.png',coin_bin, cmap='gray')
#Get connected components
cc_coins = connected_component(coin_bin)
#Print('Connected components of coins')
#plt.imshow(cc_coins)
plt.imsave('p2-1b_cc_coins.png',cc_coins)

# Part 2.2
#Tape, post-it, rectangle
tape = misc.imread('p2-2_tape.jpg', flatten=True)
post = misc.imread('p2-2_post.jpg', flatten=True)
rect = misc.imread('p2-2_rect.jpg', flatten=True)
#plt.imshow(tape)
#plt.imshow(post)
#plt.imshow(rect)
tape_bin = otsu(tape)
plt.figure()
post_bin = otsu(post)
plt.figure()
rect_bin = otsu(rect)
plt.figure()
#Downscale img to 1/10 size
tape_bin = cv2.resize(tape_bin, (0,0), fx=0.1, fy=0.1)
post_bin = cv2.resize(post_bin, (0,0), fx=0.1, fy=0.1)
rect_bin = cv2.resize(rect_bin, (0,0), fx=0.1, fy=0.1)
cc_tape = connected_component(tape_bin)
cc_post = connected_component(post_bin)
cc_rect = connected_component(rect_bin)
plt.imsave('p2-2_cctape.png', cc_tape)
plt.imsave('p2-2_ccpost.png', cc_post)
plt.imsave('p2-2_ccrect.png', cc_rect)
#plt.imshow(cc_tape)
#plt.imshow(cc_post)
#plt.imshow(cc_rect)

# Part 2.3
plot_img_moments(cc_tape,2,name='cctape',plot_align=0)
plot_img_moments(cc_post,2,name='ccpost',plot_align=0)
plot_img_moments(cc_rect,6,name='ccrect',plot_align=0)

# Part 2.4
plot_img_moments(cc_tape,2,name='cctape',plot_align=1)
plot_img_moments(cc_post,2,name='ccpost',plot_align=1)
plot_img_moments(cc_rect,6,name='ccrect',plot_align=1)



## Problem 3: Filtering

In this problem we will play with convolution filters. Filters, when convolved with an image, will respond strongest on locations of an image that look like the filter when it is rotated by 180 degrees. This allows us to use filters as object templates in order to identify specific objects within an image. In the case of this assignment, we will be finding cars within an image by convolving a car template onto that image. Although this is not a very good way to do object detection, this problem will show you some of the steps necessary to create a good object detector. The goal of this problem will be to teach some pre-processing steps to make vision algorithms be successful and some strengths and weaknesses of filters. Each problem will ask you to analyze and explain your results. If you do not provide an explanation of why or why not something happened, then you will not get full credit.

$\textbf{3.1 Warmup - Mickey Detection}$

First you will convolve a filter to a synthetic image. The filter or template is filter.jpg and the synthetic image is toy.png. These files are available on the course webpage. You will first want to modify the filter image and original slightly. We will do so by subtracting the mean of the image intensities from the image i.e. $I_t \rightarrow I - means(\textbf{vec}(I))$, where $I_t$ is the transformed image and $I$ is
the original image. 

To convolve the filter image with the toy example, use scipy.ndimage.convolve. The output of the convolution will create an intensity image. In the original image (not the image with its mean subtracted), draw a bounding box of the same size as the filter image around the top 3 intensity value locations in the convolved image. Provide both the intensity map and bounding box images in the report.

<img src="toy_conv.png">
<center>Figure 4: Example outputs for the synthetic example. Left - Intensity map. Right - Bounding boxes</center>

The outputs should look like the above figure. Describe how well you think this technique will work on more realistic images. Do you foresee any problems for this algorithm on more realistic images?

$\textbf{3.2 Detection quality}$

We have now created an algorithm that produces a bounding box around a detected object. However, we have no way to know if the bounding box is good or bad. In the example images shown above, the bounding boxes look reasonable, but not perfect. Given a ground truth bounding box (g) and a
predicted bounding box (p), a commonly used measurement for bounding box quality is:

\begin{eqnarray}
r = \frac{p \cap g}{p \cup g}
\end{eqnarray}

More intuitively, this is the number of overlapping pixels between the bounding boxes divided by the total number of unique pixels of the two bounding boxes combined. Assuming that all bounding boxes will be axis-aligned rectangles, implement this error function and try it on the toy example in the previous section. Choose 3 different ground truth bounding box sizes around one of the Mickey 3 silhouettes. In general, if the overlap is 50% or more, you may consider that the detection did a good job.

$\textbf{3.3 Car Detection}$

Now that you have created an algorithm for matching templates and a function to determine the quality of the match, it is time to try some more realistic images. The file, cartemplate.jpg , will be the filter to convolve on each of the 5 other car images $\textbf{(car1.jpg, car2.jpg, car3.jpg)}$. Each image will have an associated text files that contains two $x, y$ coordinates (one pair per line). These coordinates will be the ground truth bounding box for each image. For each car image, provide the
following:

1. A heat map image
2. The provided ground truth bounding box drawn on the original image (green)
3. The detected bounding box drawn on the original image (blue)
4. The overlap between the two bounding boxes drawn on the original image (purple)
5. The bounding box overlap percent r × 100%

<img src="car_conv.png">
<center>Figure 4: Example outputs for the car example. Left - Intensity map. Right - Bounding boxes</center>

Here are some helpful hints to increase the overlap percentage:
- Rescaling the car template to various sizes (for $\textbf{car1.jpg}$)
- Horizontally flipping the car template (for $\textbf{car2.jpg}$)
- A combination of the first two hints (for $\textbf{car3.jpg}$)
- Gaussian blurring might be useful in all cases

An example output is shown for $\textbf{car1.jpg}$ in Figure 5. It may not be possible to achieve 50% overlap on all the images. Your analysis of the images will be worth as much as achieving a high overlap percentage.

## 3.1  
Given a Mickey filter/template and a toy image with 3 Mickeys, the filter was convolved to the toy image to detect the 3 Mickeys, as seen in the intensity map of the toy image with the red circular areas. Bounding boxes were then placed over the areas in the toy image that have the closest matches to the filter.  
<img src="p3-1_intensitymap.png" />  
<img src="p3-1_boundingboxes.png" />   
  
When applied on more realistic images, the accuracy of this technique would decrease and lead to false detections since there is more noise and objects that could look similar to the filter image. 

## 3.2  
Three different ground truth bounding boxes were chosen to test bounding box qualities and to determine the overlap percentage when detecting the Mickey on the upper right hand side of the toy image. In each image, the blue box is the true bounding box, the green box is for the ground truth, and the purple shows the overlap between the two boxes. All have high overlap percentages, so the detection quality is good.  
<img src="p3-2_groundtruth0.png" />  
<img src="p3-2_groundtruth1.png" />  
<img src="p3-2_groundtruth2.png" />  
  
## 3.3  
A car filter/template was used to detect the cars in 3 different realistic car images. The filter image was downscaled before it was convolved to the car images so that it is more similar to the size of the car to be detected. The detected bounding box size is the same as the size of the filter used for each car image.  
  
Car 1 was easily detected with at least 50% overlap between the filter and the car. The color and position of the car in this image is very similar to the filter, so no further filter modifications were needed.  
<img src="p3-3_intmap_car1.png" />  
<img src="p3-3_boundbox_car1.png" />  
  
Car 2 required for the filter image to be flipped horizontally to match the orientation of car 2. The overlap is slightly more than 50%.  
<img src="p3-3_intmap_car2.png" />  
<img src="p3-3_boundbox_car2.png" />  
  
Car 3 required the most filter modifications experimentations since car 3 was difficult to detect. Car 3 is in a different color and much different orientation where mainly only the back of the car could be seen. Cropping the filter image to get the car wheels produced the most accurate detection of the tried modifications (combinations of flipping, blurring, scaling, and cropping) at around 16% since the wheels are similar in both filter and car image. Without the cropping, the detected bounding box would be placed over the tree since the color intensity and shape is roughly similar to the filter. If the filter was cropped to get the car wheels and Gaussian blurring was applied to blur the filter, the detected bounding box would move to the area on the right side of the car where the poles are in front of the black building wall since blurring would make the filter look less like wheels and more like that building wall. To increase the detection accuracy, the filter image could be inverted to match the color of car 3.  
<img src="p3-3_intmap_car3.png" />  
<img src="p3-3_boundbox_car3.png" />  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#from scipy.ndimage.filters import convolve as conv
import scipy.ndimage as ndi
import matplotlib
#import skimage.io 
from scipy import misc
import copy
import cv2

"""
def detection_quality(predicted_bbox, true_bbox):
    # your code here
    return r
"""

#find overlap area (%), and its bound box
def detection_quality(predicted_bbox, true_bbox): 
    # your code here
    a = predicted_bbox
    b = true_bbox
    #find overlap_area(a, b):  # returns None if rectangles don't intersect
    dx = min(a[2], b[2]) - max(a[0], b[0])
    dy = min(a[3], b[3]) - max(a[1], b[1])
    if (dx>=0) and (dy>=0):
        c = [max(a[0], b[0]), max(a[1], b[1]), min(a[2], b[2]), min(a[3], b[3])]
        inter_area = dx*dy
    else:
        c = []
        inter_area = 0
    print("overlap area", inter_area)
    union_area = (b[2]-b[0])*(b[3]-b[1]) + (a[2]-a[0])*(a[3]-a[1]) - inter_area
    ov_per = 100.*(inter_area / union_area)
    print("union area", union_area)
    print("bounding box overlap percent", ov_per, "%")  
    return c

'''
Part 3.1
'''
print("<*Part 3.1>:")
#grey_image = mpimg.imread('toy.png')   #  pixels
#image_toy = otsu(grey_image)
#Convert filter and image to greyscale
grey_toy = misc.imread('toy.png', flatten=True)
image_toy = grey_toy          #no need for otsu, since it is already in binary form
#print("greybinary toy", image_toy[72,:])
#im = Image.open('filter.jpg')
#im.save('filter.png')          #convert jpg to png
grey_image = misc.imread('filter.jpg', flatten=True)    # pixels
#image_filter = otsu(grey_image)
image_filter = grey_image       #no need for otsu, since it is already in binary form

#Vectorize matrix, convert elements to float, subtract global mean for each element in filter and img
#Toy
row, col = np.shape(image_toy)
#Convert matrix elements to float
image_toy = image_toy.astype(float)
toy_mean = np.mean(image_toy)
#Subtract mean from every intensity pixel
conv_float = lambda x: x-toy_mean
float_mean = np.vectorize(conv_float)
image_toy = float_mean(image_toy)

#Filter
row, col = np.shape(image_filter)
#Convert matrix elements to float
image_filter = image_filter.astype(float)
filter_mean = np.mean(image_filter)
#Subtract mean from every intensity pixel
conv_float = lambda x: x-filter_mean
float_mean = np.vectorize(conv_float)
image_filter = float_mean(image_filter)
        
#Rotate filter 180 degrees
image_filter = np.rot90(image_filter,1,(0,1))
image_filter = np.rot90(image_filter,1,(0,1))
plt.imshow(image_toy)
plt.show()
plt.imshow(image_filter)
plt.show()

#Convolve filter wih toy and get intensity map
conv_image = ndi.convolve(image_toy, image_filter, mode='constant', cval=0.0)
#print("conv toy", conv_image[72,:])
#plt.imshow(image_toy)    
#plt.show()
#plt.imshow(image_filter)    
#plt.show()
plt.imshow(conv_image, cmap='jet')    
#plt.show()
#plt.imsave('p3-1_intensitymap.png',conv_image,cmap='jet')

#Plot bounding boxes around the 3 Mickey's
heats = np.argwhere(conv_image == np.max(conv_image))
print('heats: ', heats)
print("max intensity pixel", np.max(conv_image), np.argwhere(conv_image == np.max(conv_image)))
#max intensity pixel -2159.0427 at [[115 307] [211 110]  [289 342]] 
#plt.plot([heats[0][1]], [heats[0][0]], 'ro')
#plt.plot([heats[1][1]], [heats[1][0]], 'ro')
#plt.plot([heats[2][1]], [heats[2][0]], 'ro')
row_f,col_f = np.shape(image_filter)
import matplotlib
#fig = plt.figure()
#ax = fig.add_subplot(111)
fig,ax = plt.subplots(1)
#ax.imshow(binary_image_toy)
ax.imshow(grey_toy, cmap='gray')
rect1 = matplotlib.patches.Rectangle((heats[0][1]-(col_f/2.), heats[0][0]-(row_f/2.)), col_f, row_f, edgecolor='blue', fc='none')
rect2 = matplotlib.patches.Rectangle((heats[1][1]-(col_f/2.), heats[1][0]-(row_f/2.)), col_f, row_f, edgecolor='blue', fc='none')
rect3 = matplotlib.patches.Rectangle((heats[2][1]-(col_f/2.), heats[2][0]-(row_f/2.)), col_f, row_f, edgecolor='blue', fc='none')
ax.add_patch(rect1)
ax.add_patch(rect2)
ax.add_patch(rect3)
plt.savefig('p3-1_boundingboxes.png')
plt.show()

'''
Part 3.2
'''
print("<*Part 3.2>:")
"""
fp = open("mickey1.txt", "r")
#260 40
#400 150
data = fp.readlines()
t = []
for line in data:           #file dtata read in order as 'ymin,xmin,ymax,xmax'
    t.append(line.split())
    #print(t)
ymin = int(t[0][0])
xmin = int(t[0][1])
ymax = int(t[1][0])
xmax = int(t[1][1])
b = [xmin,ymin,xmax,ymax]
"""
b = [40,260,150,400],[30,270,150,400],[20,270,150,400]
#Plot 3 ground truths
for i in range(0,len(b)):
    #pick the top right Mickey
    fig,ax = plt.subplots(1)
    ax.imshow(grey_toy, cmap='gray')    
    rect1 = matplotlib.patches.Rectangle((heats[0][1]-(col_f/2.), heats[0][0]-(row_f/2.)), col_f, row_f, edgecolor='blue', fc='none')
    ax.add_patch(rect1)

    #Rectangle as 'xmin ymin xmax ymax'
    a = [heats[0][0]-(row_f/2.), heats[0][1]-(col_f/2.), heats[0][0]+(row_f/2.), heats[0][1]+(col_f/2.)] 
    print("detected box", a)  #[24.5, 276.5, 151.5, 403.5]

    print("truth box", b[i])
    rectg = matplotlib.patches.Rectangle((b[i][1], b[i][0]), (b[i][3]-b[i][1]), (b[i][2]-b[i][0]), edgecolor='green', fc='none')
    ax.add_patch(rectg)

    c = detection_quality(a, b[i]) #find overlap area and detect quality in %
    #print("overlap box", c)
    if(c != []):
        rectv = matplotlib.patches.Rectangle((c[1], c[0]), (c[3]-c[1]), (c[2]-c[0]), edgecolor='purple', fc='none')
        ax.add_patch(rectv)
    plt.savefig('p3-2_groundtruth' + str(i) + '.png')
    plt.show()


'''
Part 3.3
'''
print("<*Part 3.3>:")
#Convert filter to greyscale
grey_filter = misc.imread('cartemplate.jpg', flatten=True)
image_filter = grey_filter

#Vectorize matrix, convert elements to float, subtract global mean for each element in filter 
#Filter
row, col = np.shape(image_filter)
#Convert matrix elements to float
image_filter = image_filter.astype(float)
filter_mean = np.mean(image_filter)
#Subtract mean from every intensity pixel
conv_float = lambda x: x-filter_mean
float_mean = np.vectorize(conv_float)
image_filter = float_mean(image_filter)
        
#Rotate filter 180 degrees
image_filter = np.rot90(image_filter,1,(0,1))
image_filter = np.rot90(image_filter,1,(0,1))
plt.imshow(image_filter)
plt.show()

#3 cars
for i in range(1,4):
    #Downscale filter
    if i == 1:
        img_filter = cv2.resize(image_filter, (0,0), fx=1/8, fy=1/8)
    #For cars 2 and 3
    else:
        if i == 2:
            img_filter = cv2.resize(image_filter, (0,0), fx=1/6, fy=1/6)
            #Flip filter horizontally
            img_filter = np.flip(img_filter, 1)   
        if i == 3:
            img_filter = cv2.resize(image_filter, (0,0), fx=1/6, fy=1/6)
            #Crop to get wheels 
            row, col = np.shape(img_filter)
            img_filter = img_filter[:int(row*2/4),:]
            #Apply Gaussian blur to filter for car 3
            #img_filter = ndi.gaussian_filter(img_filter, sigma=1)
    print('  <*Car' + str(i) + '>:')
    #Convert car img to greyscale
    grey_car = misc.imread('car' + str(i) + '.jpg', flatten=True)
    image_car = grey_car
    
    #Vectorize matrix, convert elements to float, subtract global mean for each element in car
    #Car
    row, col = np.shape(image_car)
    #Convert matrix elements to float
    image_car = image_car.astype(float)
    car_mean = np.mean(image_car)
    #Subtract mean from every intensity pixel
    conv_float = lambda x: x-car_mean
    float_mean = np.vectorize(conv_float)
    image_car = float_mean(image_car)
    #plt.imshow(image_car, cmap = 'gray')
    #plt.show()
    
    #Convolve filter wih car and get intensity map
    conv_image = ndi.convolve(image_car, img_filter, mode='constant', cval=0.0)
    #plt.imshow(image_car)    
    #plt.show()
    plt.imshow(img_filter)    
    plt.show()
    plt.imshow(conv_image, cmap='jet')    
    #plt.show()
    plt.imsave('p3-3_intmap_car' + str(i) + '.png',conv_image,cmap='jet')
    
    #Plot bounding boxes around car
    heats = np.argwhere(conv_image == np.max(conv_image))
    print('max intensity pixel', np.max(conv_image), np.argwhere(conv_image == np.max(conv_image)))
    row_f,col_f = np.shape(img_filter)
    fig,ax = plt.subplots(1)
    ax.imshow(grey_car, cmap='gray')    
    rect1 = matplotlib.patches.Rectangle((heats[0][1]-(col_f/2.), heats[0][0]-(row_f/2.)), col_f, row_f, edgecolor='blue', fc='none')
    ax.add_patch(rect1)
    #plt.show()
    
    #Rectangle as 'xmin ymin xmax ymax'
    a = [heats[0][0]-(row_f/2.), heats[0][1]-(col_f/2.), heats[0][0]+(row_f/2.), heats[0][1]+(col_f/2.)] 
    print('detected box', a)  #[122.0, 252.0, 242.0, 474.0]
    
    fp = open('car' + str(i) + '.txt', 'r')
    data = fp.readlines()
    t = []
    for line in data:
        t.append(line.split())
        #print(t)
    ymin = int(t[0][0])
    xmin = int(t[0][1])
    ymax = int(t[1][0])
    xmax = int(t[1][1])
    b = [xmin,ymin,xmax,ymax]
    print("truth box", b)
    rectg = matplotlib.patches.Rectangle((b[1], b[0]), (b[3]-b[1]), (b[2]-b[0]), edgecolor='green', fc='none')
    ax.add_patch(rectg)
    
    c = detection_quality(a, b) #find overlap area and detect quality in %
    #print("overlap box", c)
    if(c != []):
        rectv = matplotlib.patches.Rectangle((c[1], c[0]), (c[3]-c[1]), (c[2]-c[0]), edgecolor='purple', fc='none')
        ax.add_patch(rectv)
    plt.savefig('p3-3_boundbox_car' + str(i) + '.png')
    plt.show()
    